In [3]:
import pandas as pd
import numpy as np
import seaborn  as sns
import matplotlib.pyplot as plt
import math
import time as t
from pylab import rcParams
from sklearn import tree
%matplotlib inline
%config IPCompleter.greedy=True

In [4]:
def readData():
    """
    load data from the file path
    factory function
    """
    #data_long = pd.read_excel("/Users/dilruba_p/Desktop/dataset_576_long.xlsx") #Dilruba mac
    data_long = pd.read_excel("/Users/haominshi/Desktop/al_data/dataset_576_long.xlsx") #Haomin mac
    #data_long = pd.read_excel("/home/haomin/Desktop/bio-data/dataset_576_long.xlsx") # thinkpad ubuntu
    df = data_long.copy()
    print("Finish Reading Data /dataset_576_long.xlsx")
    return df

def removeNoise(df):
    """
    Argument = df
    return = df
    
    drop some noise of the that might pollute the result
    """
    #medications with analgesic_rx
    #df = df[df.analgesic_rx !=1]
    #removing people who have a suspected of having head injuries in past visits 
    #df = df[df.headinjrloc_cum !=1]
    #removing people who have a suspected of having cancer in past visits 
    #df = df[df.cancer_cum !=1]
    #removing people who have a suspected of having diabetes in past visits 
    df = df[df.dm_cum !=1]
    print(df.shape)
    return df

def groupbyid(df):
    """
    arg = df
    return = list of df
    
    convert the df to a list of df that is grouped by the patient id
    return a list of df
    """
    # convert the values from the projid to a list
    projid_all = df.projid.unique()
    projid_list = projid_all.tolist() # a unique list of ID  acquired 

    patient_dfbyid = []
    start_df_row = 0 # selecting the first row 
    end_df_row   = 0 # selecting the last row
    flag_start   = 0 # start index for df spliting based on unique ID
    df = df.reset_index(drop = True) # df reindexed

    for p_id in projid_all: # loop through the unique id list
        for df_index in range(flag_start, df.shape[0]): 
            if df.iloc[df_index]['projid'] == p_id: end_df_row += 1
            elif df.iloc[df_index]['projid'] != p_id: break
        patient_dfbyid.append(df[start_df_row: end_df_row])
        start_df_row = end_df_row
        flag_start = end_df_row

    return patient_dfbyid # a list of df, that each element of the list is one patient

def patient_diagnosis(list_of_patient):
    """
    arg = list
    return list
    
    This function split the patient list to 2 sub list, each of them contain a whole DF for that patient
    The difference is that one list is for patient that is diagonsed in future, the other is not
    it is splif via the dcfdx value, dcfdx >= 3 means diagnosed
    """

    patient_dfbyid_diagnosed = []      #  diagnosed with alzheimer
    patient_dfbyid_not_diagnosed = []  # not diagnalzheimer
    diagonosed = False

    for patient in list_of_patient:
        dcfdx_list = patient['dcfdx'].tolist()
        for dcfdx_element in dcfdx_list:
            if dcfdx_element >= 3: # >= 4 means diagnoised
                diagonosed = True
                break
        if (diagonosed) == True:
            patient_dfbyid_diagnosed.append(patient)
            diagonosed= False
        else: patient_dfbyid_not_diagnosed.append(patient)
    
    return patient_dfbyid_diagnosed, patient_dfbyid_not_diagnosed

def appending_initial_df_to_full_df(list_of_df):
    """
    argument = list
    return = df
    
    This list takes in a list of dataframe and appending the first df to achieve concat, it returns
    the first df at the end
    """
    list_of_df_copy = list_of_df[:]
    counter = 0
    for df in list_of_df_copy[1:]:
        counter+=1
        print("appending{}".format(counter))
        list_of_df_copy[0] = list_of_df_copy[0].append(df, ignore_index=True)
    return list_of_df_copy[0]

def imputemean_per_id(list_of_df_temp,focus = "crn"):
    for df in list_of_df_temp:
        M = df[focus].mean()
        df[focus].fillna(M,inplace=True)
    return list_of_df_temp

def some_cleaning(listOfDf):
    """
    with special focus on certain columns, if any of these columns in the data is nan
    the row with that column is dropped, column comes form the string in the method
    
    argumnet = list
    return = list
    """
    feature_str = \
    "chlstrl gfr_mdrs glucose hdlchlstrl ldlchlstrl cogn_global cts_fruits cts_idea cts_sdmt iadlsum katzsum rosbscl rosbsum phys5itemsum"
    label = "dcfdx"
    
    #feature_str = \
    #"gfr_mdrs cogn_global cts_fruits cts_idea cts_sdmt iadlsum rosbsum"
    #label = "dcfdx"
    
    feature_item_list = feature_str.split(" ")
    # only pick rows that have these values
    test_list_patient = patient_visit_list[:]
    for df in test_list_patient:
        for feature in feature_item_list:
            df = df[np.isfinite(df[feature])]
    return test_list_patient

def fillNaNs(list_of_df):
    for df in list_of_df:
        pass
    


In [5]:
raw_dataframe = readData() # load excel to df
print("Done 1")
print(raw_dataframe.shape)
noise_removed_dataframe = removeNoise(raw_dataframe) # remove some noise
print("Done 2")
patient_visit_list = groupbyid(noise_removed_dataframe) # put df to a list grouped by patient id
print("Done 3")
patient_diagnosed, patient_not_diagnosed = patient_diagnosis(patient_visit_list)
print("{} amount people diagnosed, {} amount of people not diagnosed"\
      .format(len(patient_diagnosed), len(patient_not_diagnosed)))
print("{} total patient amount".format(len(patient_visit_list)))
print("Cell finished")

print(noise_removed_dataframe.shape)

Finish Reading Data /dataset_576_long.xlsx
Done 1
(25570, 203)
(21254, 203)
Done 2
Done 3
841 amount people diagnosed, 2048 amount of people not diagnosed
2889 total patient amount
Cell finished
(21254, 203)


In [6]:
list_of_columns = ['chlstrl','crn','gfr_mdrs','glucose','hba1c','hdlchlstrl', 'hdlratio','hemacrit','hemoglbn',
'ldlchlstrl', 'mch', 'mchc', 'mcv', 'platelet','rbc', 'dcfdx', 'projid', 'age_at_visit'] 
# these are the columns that we are interesed in, However, right now i dont have a good idea about what value I
# should be use to replace the NaN

__One of the approach we have is to replace the NaNs with local mean of that column value based on that patient's visit. Personally, I dont think that is a good idea. Certainly, if a patient have dcfdx span from 1 to 4, the mean then lost its meanings.__

Check the diabieties, how?
1. Person is in first visit, and he/she does not have AD and D
2. Person is in second visit, current they have dcfdx == 4, we want to see next visit if they have D
3. Person is in third or more visit, with AD already diagnosed check if they have D

Some special trails:
1. I tried not to drop anyone who is diagnosed with Ob but this seems not really affect the outcome, even tho the original sample size is much larger

In [7]:
# some experiments
print("{} original len".format(len(patient_visit_list)))
def check_dcfdx_dP_clean_1(patient_list_id):
    """
    check if there is a relation between poeple who have AD and D
    bmi 25, 30 this is overweight
    bmi 30, 40 this is ob
    bmi 40 + this is very ob
    """
    return_list_of_patent = []
    for patient in patient_list_id:
        if patient.iloc[0]['dcfdx'] == 1 and patient.iloc[0]['bmi'] < 25:
            return_list_of_patent.append(patient)
            continue # skip to next patient
    return return_list_of_patent

def check_dcfdx_dp_clean_2(patient_list_id):
    return_list_of_patent = []
    for patient in patient_list_id:
        for row_index in range(patient.shape[0]):
            if patient.iloc[row_index]['dcfdx'] >= 4 and patient.iloc[row_index]['bmi'] >= 25:
                return_list_of_patent.append(patient)
                break # no need to scan if found
    return return_list_of_patent

def check_dcfdx_dp_clean_special(patient_list_id):
    return_list_of_patent = []
    for patient in patient_list_id:
        for row_index in range(patient.shape[0]):
            if patient.shape[0] < 3:
                break # dont care about patient has less visit than 3
            if patient.iloc[row_index]['dcfdx'] >= 4 and patient.iloc[row_index]['bmi'] >= 25:
                return_list_of_patent.append(patient)
                break # no need to scan if found
    return return_list_of_patent


test_list_stage1 = check_dcfdx_dP_clean_1(patient_visit_list)
print("{} stage 1 clean".format(len(test_list_stage1)))
test_list_stage2 = check_dcfdx_dp_clean_2(test_list_stage1)
print("{} stage 2 clean".format(len(test_list_stage2)))
test_list_stage_special = check_dcfdx_dp_clean_special(patient_visit_list)
print("{} stage 3 clean".format(len(test_list_stage_special)))
            

2889 original len
702 stage 1 clean
33 stage 2 clean
349 stage 3 clean


In [8]:
focus_patient_list = some_cleaning(patient_visit_list)

In [9]:
# Calculate counter and the percentage of ppl that is sick
counter = 0

for patient in focus_patient_list:
    for row_number in range(patient.shape[0]):
        if patient.iloc[row_number]['dcfdx'] >= 4:
            counter+=1
            break
                
print("there are total {} patient, within that there are {} people was dcfdx >= 4 ignore previous record, \n that is {} %"\
     .format(len(focus_patient_list), counter, counter/len(focus_patient_list) * 100))

counter = 0
for patient in focus_patient_list:
    for row_number in range(patient.shape[0]):
        if patient.iloc[0]['dcfdx'] == 2:
            if patient.iloc[row_number]['dcfdx'] >= 4:
                counter+=1
                break
print("there are total {} patient, within that there are {} people was dcfdx <= 3 to dcfdx >= 4, \n that is {} %"\
     .format(len(focus_patient_list), counter, counter/len(focus_patient_list) * 100))

counter = 0
for patient in focus_patient_list:
    for row_number in range(patient.shape[0]):
        if patient.iloc[0]['dcfdx'] == 1:
            if patient.iloc[row_number]['dcfdx'] >= 4:
                counter+=1
                break
print("there are total {} patient, within that there are {} people was dcfdx == 1 to dcfdx >= 4, \n that is {} %"\
     .format(len(focus_patient_list), counter, counter/len(focus_patient_list) * 100))

# some explore

counter = 0
for patient in focus_patient_list:
    for row_number in range(patient.shape[0]):
        if patient.shape[0] > 2:
            if patient.iloc[0]['dcfdx'] <= 2 or patient.iloc[1]['dcfdx'] <= 2 or patient.iloc[2]['dcfdx'] <= 2:
                if patient.iloc[row_number]['dcfdx'] >= 4:
                    counter+=1
                    break
print("there are total {} patient, within that there are {} in explore function, \n that is {} %"\
     .format(len(focus_patient_list), counter, counter/len(focus_patient_list) * 100))



there are total 2889 patient, within that there are 818 people was dcfdx >= 4 ignore previous record, 
 that is 28.314295604015232 %
there are total 2889 patient, within that there are 283 people was dcfdx <= 3 to dcfdx >= 4, 
 that is 9.795777085496711 %
there are total 2889 patient, within that there are 355 people was dcfdx == 1 to dcfdx >= 4, 
 that is 12.287988923502942 %
there are total 2889 patient, within that there are 653 in explore function, 
 that is 22.602976808584284 %


### Train prep
Reuse the previous functions from the last cell, I will construct 3 dataframes, each of the dataframe represent one possible route:
1. not diagnosed (past) -> diagnosed (now)
2. not diagnosed (past) -> not diagnosed (now)
3. diagnosed (past) -> diagnosed (now)

First, construct the dataframe, than use 70% of each of the df to train, 20% to test, 10% to validate, __current plan__

In [13]:
def create3dfs (focus_patient_list):
    work_list_this_fn = focus_patient_list[:] # set up
    # first loop to concat the nd - nd
    feature_str = \
    "projid dcfdx chlstrl gfr_mdrs glucose hdlchlstrl ldlchlstrl cogn_global cts_fruits cts_idea cts_sdmt iadlsum katzsum rosbscl rosbsum phys5itemsum"
    # label = "dcfdx"
    feature_str_list = feature_str.split(" ")
    df_nd_nd = pd.DataFrame(columns=feature_str_list) # new df with only the feature str created
    df_nd_d = pd.DataFrame(columns=feature_str_list)
    df_d_d = pd.DataFrame(columns=feature_str_list)
    
    # something WRONG with this LOOP !!!!!
    for patient in work_list_this_fn: # ND ND
        flag = True
        for row_number in range(patient.shape[0]):
            if patient.iloc[row_number]['dcfdx'] > 1:
                Flag = False
                break
        if flag == True:
            df_nd_nd = df_nd_nd.append(patient[feature_str_list])
            print(patient['dcfdx'])
        elif flag == False:
            continue
        
                
    print("df_nd_nd DONE!")
    for patient in work_list_this_fn: # ND D
        for row_number in range(patient.shape[0]):
            if patient.shape[0] > 2:
                if patient.iloc[0]['dcfdx'] <= 2 \
                or patient.iloc[1]['dcfdx'] <= 2 \
                or patient.iloc[2]['dcfdx'] <= 2:
                    if patient.iloc[row_number]['dcfdx'] >= 4:
                        df_nd_d = df_nd_d.append(patient[feature_str_list])
    print("df_nd_d DONE!")
    for patient in work_list_this_fn: # D D
        if patient.iloc[0]['dcfdx'] > 3:
            df_d_d = df_d_d.append(patient[feature_str_list])
    print("df_d_d DONE!")
                
    return df_nd_nd, df_nd_d, df_d_d

train_nd_nd, train_nd_d, train_d_d = create3dfs(focus_patient_list)

def removeNanRow(df):
    """remove all the rows that has Nan in it"""
    feature_str = \
    "projid dcfdx chlstrl gfr_mdrs glucose hdlchlstrl ldlchlstrl cogn_global cts_fruits cts_idea cts_sdmt iadlsum katzsum rosbscl rosbsum phys5itemsum"
    # label = "dcfdx"
    feature_str_list = feature_str.split(" ")
    for feature in feature_str_list:
        df = df[np.isfinite(df[feature])]
    return df

train_nd_nd = removeNanRow(train_nd_nd)
train_nd_d = removeNanRow(train_nd_d)
train_d_d = removeNanRow(train_d_d)




print(train_nd_nd['dcfdx'])
#print(train_nd_d.head())
#print(train_d_d.head())




0    1.0
1    1.0
2    1.0
3    1.0
4    1.0
5    1.0
6    1.0
7    1.0
Name: dcfdx, dtype: float64
8     4.0
9     4.0
10    4.0
Name: dcfdx, dtype: float64
11    1.0
Name: dcfdx, dtype: float64
12    1.0
13    1.0
14    1.0
Name: dcfdx, dtype: float64
15    1.0
16    1.0
17    1.0
18    1.0
19    1.0
20    1.0
21    1.0
22    1.0
23    1.0
Name: dcfdx, dtype: float64
24    1.0
25    2.0
26    1.0
27    1.0
28    2.0
29    4.0
30    4.0
31    4.0
32    2.0
33    NaN
34    4.0
35    4.0
Name: dcfdx, dtype: float64
36    2.0
37    1.0
38    1.0
39    1.0
40    1.0
Name: dcfdx, dtype: float64
41    4.0
42    4.0
Name: dcfdx, dtype: float64
43    1.0
44    1.0
45    1.0
Name: dcfdx, dtype: float64
46    1.0
47    1.0
Name: dcfdx, dtype: float64
48    1.0
49    1.0
50    1.0
51    1.0
52    1.0
53    1.0
54    1.0
55    1.0
56    1.0
57    1.0
58    1.0
59    1.0
Name: dcfdx, dtype: float64
60    2.0
61    4.0
62    4.0
63    4.0
64    4.0
65    4.0
Name: dcfdx, dtype: float64
66    1.0
67

841    1.0
842    2.0
843    2.0
844    2.0
Name: dcfdx, dtype: float64
845    4.0
846    4.0
847    4.0
848    4.0
Name: dcfdx, dtype: float64
849    1.0
850    1.0
851    1.0
852    1.0
Name: dcfdx, dtype: float64
853    1.0
854    1.0
855    1.0
856    1.0
857    2.0
858    2.0
Name: dcfdx, dtype: float64
859    1.0
860    1.0
861    1.0
862    1.0
863    1.0
864    1.0
865    1.0
866    1.0
867    1.0
868    1.0
869    1.0
870    1.0
871    2.0
Name: dcfdx, dtype: float64
872    1.0
873    1.0
874    1.0
875    1.0
876    1.0
Name: dcfdx, dtype: float64
877    2.0
878    1.0
879    2.0
880    2.0
Name: dcfdx, dtype: float64
881    2.0
882    1.0
883    1.0
884    1.0
885    1.0
886    1.0
887    1.0
888    1.0
889    1.0
890    1.0
891    1.0
Name: dcfdx, dtype: float64
892    1.0
893    1.0
894    2.0
895    1.0
896    1.0
897    1.0
898    1.0
899    2.0
900    1.0
901    1.0
902    1.0
903    1.0
Name: dcfdx, dtype: float64
904    1.0
905    1.0
906    1.0
907    1.0
908    1.0


1712    1.0
1713    1.0
1714    1.0
1715    1.0
1716    1.0
1717    1.0
1718    1.0
1719    1.0
1720    1.0
1721    1.0
1722    1.0
1723    1.0
1724    1.0
1725    1.0
1726    1.0
Name: dcfdx, dtype: float64
1727    2.0
1728    2.0
Name: dcfdx, dtype: float64
1729    1.0
1730    1.0
1731    1.0
1732    1.0
1733    1.0
1734    1.0
1735    1.0
1736    1.0
1737    2.0
Name: dcfdx, dtype: float64
1738    1.0
1739    1.0
1740    1.0
1741    1.0
Name: dcfdx, dtype: float64
1742    1.0
Name: dcfdx, dtype: float64
1743    1.0
1744    1.0
1745    1.0
1746    1.0
1747    1.0
1748    1.0
1749    1.0
1750    1.0
Name: dcfdx, dtype: float64
1751    1.0
1752    1.0
1753    1.0
Name: dcfdx, dtype: float64
1754    4.0
1755    4.0
1756    4.0
1757    4.0
Name: dcfdx, dtype: float64
1758    2.0
1759    2.0
1760    1.0
Name: dcfdx, dtype: float64
1761    1.0
1762    1.0
1763    1.0
1764    1.0
1765    1.0
1766    1.0
1767    1.0
1768    1.0
1769    1.0
1770    1.0
1771    1.0
1772    1.0
1773    1.0
1774

2335    1.0
2336    1.0
2337    1.0
2338    1.0
2339    1.0
2340    1.0
2341    1.0
2342    1.0
2343    1.0
2344    1.0
2345    1.0
2346    1.0
2347    1.0
2348    1.0
2349    1.0
2350    1.0
Name: dcfdx, dtype: float64
2351    1.0
2352    1.0
2353    1.0
2354    1.0
2355    1.0
2356    1.0
2357    1.0
2358    1.0
2359    1.0
2360    1.0
2361    1.0
2362    1.0
2363    1.0
2364    1.0
2365    1.0
2366    1.0
2367    2.0
2368    1.0
2369    2.0
2370    2.0
2371    1.0
2372    4.0
Name: dcfdx, dtype: float64
2373    2.0
2374    1.0
2375    1.0
Name: dcfdx, dtype: float64
2376    1.0
2377    1.0
2378    1.0
2379    1.0
2380    1.0
2381    1.0
2382    1.0
2383    1.0
2384    1.0
2385    2.0
2386    1.0
2387    1.0
2388    1.0
2389    1.0
2390    1.0
Name: dcfdx, dtype: float64
2391    2.0
2392    2.0
2393    2.0
2394    4.0
2395    4.0
2396    4.0
2397    4.0
2398    4.0
2399    6.0
Name: dcfdx, dtype: float64
2400    1.0
2401    1.0
2402    1.0
2403    1.0
2404    NaN
2405    1.0
2406    

3230    1.0
3231    1.0
3232    1.0
3233    1.0
3234    1.0
3235    1.0
3236    1.0
3237    1.0
3238    1.0
3239    1.0
3240    1.0
3241    1.0
3242    1.0
3243    1.0
3244    1.0
3245    1.0
Name: dcfdx, dtype: float64
3246    1.0
3247    1.0
3248    1.0
3249    1.0
3250    1.0
3251    1.0
3252    1.0
3253    1.0
3254    1.0
3255    1.0
3256    1.0
3257    1.0
3258    1.0
3259    1.0
3260    1.0
3261    1.0
3262    1.0
Name: dcfdx, dtype: float64
3263    1.0
3264    1.0
3265    1.0
3266    1.0
3267    1.0
3268    1.0
3269    1.0
3270    1.0
3271    1.0
3272    1.0
3273    1.0
3274    1.0
3275    1.0
3276    1.0
3277    1.0
3278    1.0
3279    1.0
3280    1.0
3281    1.0
3282    1.0
3283    1.0
Name: dcfdx, dtype: float64
3284    1.0
3285    1.0
3286    1.0
3287    1.0
Name: dcfdx, dtype: float64
3288    1.0
3289    1.0
3290    1.0
3291    1.0
3292    1.0
3293    1.0
3294    1.0
3295    1.0
3296    1.0
3297    1.0
3298    1.0
Name: dcfdx, dtype: float64
3299    1.0
3300    1.0
3301    

...

19466    1.0
19467    2.0
19468    2.0
19469    2.0
19470    NaN
19471    2.0
19472    2.0
19473    2.0
19474    4.0
19475    4.0
Name: dcfdx, dtype: float64
19476    1.0
19477    1.0
19478    1.0
19479    1.0
19480    NaN
Name: dcfdx, dtype: float64
19481    1.0
19482    1.0
19483    1.0
19484    1.0
19485    1.0
19486    1.0
Name: dcfdx, dtype: float64
19487    2.0
Name: dcfdx, dtype: float64
19488    1.0
19489    1.0
19490    1.0
19491    1.0
19492    1.0
Name: dcfdx, dtype: float64
19493    2.0
19494    2.0
19495    2.0
19496    2.0
Name: dcfdx, dtype: float64
19497    1.0
19498    1.0
19499    1.0
19500    1.0
19501    1.0
19502    1.0
19503    1.0
Name: dcfdx, dtype: float64
19504    5.0
19505    3.0
19506    3.0
19507    1.0
19508    5.0
19509    5.0
19510    5.0
Name: dcfdx, dtype: float64
19511    1.0
19512    1.0
19513    1.0
19514    2.0
19515    1.0
19516    1.0
19517    2.0
Name: dcfdx, dtype: float64
19518    1.0
19519    1.0
19520    1.0
19521    NaN
Name: dcfdx, dtype: 

20131    2.0
20132    4.0
Name: dcfdx, dtype: float64
20133    1.0
20134    1.0
20135    1.0
20136    1.0
20137    1.0
20138    1.0
20139    1.0
Name: dcfdx, dtype: float64
20140    1.0
Name: dcfdx, dtype: float64
20141    2.0
20142    2.0
20143    2.0
20144    4.0
Name: dcfdx, dtype: float64
20145    2.0
20146    2.0
20147    2.0
20148    4.0
Name: dcfdx, dtype: float64
20149    1.0
20150    1.0
20151    1.0
20152    1.0
20153    1.0
20154    1.0
20155    1.0
20156    1.0
20157    1.0
20158    1.0
20159    1.0
20160    1.0
20161    1.0
20162    1.0
20163    1.0
20164    NaN
Name: dcfdx, dtype: float64
20165    2.0
20166    4.0
Name: dcfdx, dtype: float64
20167    1.0
20168    1.0
20169    1.0
20170    1.0
20171    1.0
20172    1.0
20173    1.0
20174    1.0
Name: dcfdx, dtype: float64
20175    1.0
20176    1.0
20177    1.0
20178    1.0
20179    1.0
Name: dcfdx, dtype: float64
20180    1.0
20181    1.0
20182    1.0
20183    1.0
20184    1.0
20185    1.0
20186    1.0
20187    1.0
20188  

20765    1.0
20766    1.0
20767    1.0
20768    1.0
20769    1.0
20770    1.0
20771    1.0
20772    1.0
20773    1.0
20774    1.0
20775    1.0
Name: dcfdx, dtype: float64
20776    2.0
20777    2.0
20778    2.0
20779    4.0
20780    4.0
20781    4.0
Name: dcfdx, dtype: float64
20782    4.0
Name: dcfdx, dtype: float64
20783    1.0
20784    1.0
20785    2.0
Name: dcfdx, dtype: float64
20786    2.0
20787    1.0
20788    1.0
20789    1.0
20790    1.0
20791    1.0
20792    1.0
Name: dcfdx, dtype: float64
20793    1.0
20794    1.0
20795    4.0
20796    4.0
20797    2.0
20798    NaN
20799    4.0
20800    4.0
20801    4.0
Name: dcfdx, dtype: float64
20802    2.0
Name: dcfdx, dtype: float64
20803    2.0
Name: dcfdx, dtype: float64
20804    1.0
20805    1.0
20806    2.0
20807    1.0
20808    1.0
20809    1.0
20810    1.0
20811    1.0
20812    1.0
20813    2.0
20814    2.0
20815    1.0
Name: dcfdx, dtype: float64
20816    1.0
20817    1.0
20818    1.0
Name: dcfdx, dtype: float64
20819    2.0
20820

df_nd_d DONE!
df_d_d DONE!


TypeError: ufunc 'isfinite' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [35]:
a_new_list = groupbyid(train_nd_d)
#print(a_new_list[0])

def getInterval(list_df):
    alltogether = "chlstrl gfr_mdrs glucose hdlchlstrl ldlchlstrl cogn_global cts_fruits cts_idea cts_sdmt iadlsum katzsum rosbscl rosbsum phys5itemsum"\
    .split(" ")
    the_dict = {}
    newlist = []
    for patient in list_df:
        for column_names in alltogether:
            newlist.append(patient.iloc[-1][column_names] - patient.iloc[0][column_names])
        the_dict[patient.iloc[0]['projid']] = newlist
        newlist = []
    return the_dict

result = getInterval(a_new_list)
print(result[285563])
            

[nan, nan, nan, nan, nan, nan, nan, nan, nan, 7.0, 6.0, -2.0, 2.0, -5.583333333333333]
